In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
import collections
import ast
import more_itertools as mit
from Bio.Seq import Seq
from Bio import SeqIO
import os
from Bio import pairwise2
from scipy.spatial import distance
from tqdm import tqdm

/home/mark/anaconda3/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
FullLengthList=[]
#badChromosomes = ['chrY_hap2', 'chrY_hap1', 'chrX_hap1','CP068255.2','NC_060947.1','CP086569.2','NC_060948.1',
#                 'CM055469.2','CM055470.2','NC_073248.2','NC_011120.1','CM054605.2','CM054581.2',
#                'CM054457.2','NC_072421.2','CM054458.2','NC_072422.2',
#                 'CM055495.2','NC_073272.2','CM055496.2','NC_073273.2','CM054483.2','CM054507.2',
#                  'CM054702.2','NC_072008.2','CM054703.2','NC_072009.2',
#                  'CM054653.2','NC_072396.2','CM054654.2','NC_072397.2']

In [3]:
#['126 0.8 1.6 0.0 CM054630.2:98817513-98821222 33 153 (3556) + (CCCTCT)n Simple_repeat 1 123 (0) 3071', '920 10.5 9.8 28.2 CM054630.2:98817513-98821222 154 3621 (88) + SVA_B Retroposon/SVA 52 1381 (2) 3072']
#1
#Truncated

In [4]:
directory='/home/mark/Desktop/TE_Leaves/ls_primates/Final_Dataframes/Cactus/'
elementSizes={'SINE/Alu':250,'LINE/L1':5900,'Retroposon/SVA':1100}

for csvFile in os.listdir(directory):
    df=pd.read_csv(directory+csvFile, low_memory=False).set_index("ID")
    #if 'PonPyg' in str(csvFile):
    #    break
    #else:
    #    pass
    
    nonFullLengthSVAs=[]
    fullLengthAlus=[]
    fullLengthLines=[]
    fullLengthSVA=[]
    
    for row in df.index:
        
        element = str(df.at[row,'Element_Annotation'])
        teProportion = float({x.upper():y for x,y in ast.literal_eval(str(df.at[row,'TE_Proportion'])).items()}[element.upper()])
        length = int(df.at[row,'Sequence_Length'])
        #print(length, teProportion, element)

        if length*teProportion>=elementSizes[str(df.at[row,'TE_Designation'])]:
            #print(row)
                                             
            if str(df.at[row,'TE_Designation']) == 'LINE/L1':
                fullLengthLines.append(row)
                                             
            elif str(df.at[row,'TE_Designation']) == 'SINE/Alu':
                flag=0
                teHits = ast.literal_eval(str(df.at[row,'TE_Hits']))
                for hit in teHits:
                    goodHit = hit.split()
                    if goodHit[9].upper() == element.upper():
                        if goodHit[8] == 'C' and int(goodHit[12])>=267 and int(goodHit[13])<=4:
                            flag=1
                            
                        elif goodHit[8] == '+' and int(goodHit[11])<=4 and int(goodHit[12])>=267:
                            flag=1
                            
                        else:
                            continue
                        
                    else:
                        continue
                if flag==1:                        
                    fullLengthAlus.append(row)
                else:
                    continue

                    
            ############# SVA #################################
            elif str(df.at[row,'TE_Designation']) == 'Retroposon/SVA':
                flag=0
                teHits = ast.literal_eval(str(df.at[row,'TE_Hits']))

                if len(teHits)==1:
                    #print(teHits)
                    for hit in teHits:
                        goodHit = hit.split()
                        #print(goodHit)
                        if str(goodHit[10]) == 'Retroposon/SVA':
                            
                            if goodHit[8] == 'C' and int(goodHit[12])>=1336 and int(goodHit[13])<=60:
                                flag=1

                            elif goodHit[8] == '+' and int(goodHit[11])<=60 and int(goodHit[12])>=1336:
                                flag=1

                            else:
                                continue

                        else:
                            continue

                    if flag==1:                        
                        fullLengthSVA.append(row)
                    else:
                        continue
                        
                        
                else:
                    #print(teHits)
                    #print('\n')
                    
                    frontHalf = 0 
                    backHalf = 0
                    
                    for hit in teHits:
                        
                        goodHit = hit.split()
                        
                        if str(goodHit[10]) == 'Retroposon/SVA':
                            
                            if goodHit[8] == 'C' and int(goodHit[12])>=1336 and int(goodHit[13])<=60:
                                frontHalf=1
                                backHalf=1

                            elif goodHit[8] == '+' and int(goodHit[11])<=60 and int(goodHit[12])>=1336:
                                frontHalf=1
                                backHalf=1

                            elif goodHit[8] == 'C' and int(goodHit[12])>=1336:
                                backHalf=1
                            
                            elif goodHit[8] == 'C' and int(goodHit[13])<=60:
                                frontHalf=1
                                
                            elif goodHit[8] == '+' and int(goodHit[11])<=60:
                                frontHalf=1
                                
                            elif goodHit[8] == '+' and int(goodHit[12])>=1336:
                                backHalf=1
                            
                            
                            else:
                                continue

                        else:
                            continue
                            
                    #print(flag)
                    if backHalf==1 and frontHalf==1:
                        #print("FullLength")
                        #print('\n')
                        fullLengthSVA.append(row)
                    else:
                        nonFullLengthSVAs.append(row)
                        #print("Truncated")
                        #print('\n')
                        continue
                        
            ###########################################################################################
                


        else:
            continue
    
    #break
    #print(csvFile)
    speciesName = str(csvFile.split("_")[0])
    
    fullLineDF = df.loc[fullLengthLines].copy()
    fixedLINEDic = collections.Counter(fullLineDF['Fixed'])
    
    fullAluDF = df.loc[fullLengthAlus].copy()
    fixedAluDic = collections.Counter(fullAluDF['Fixed'])
    
    fullSVADF = df.loc[fullLengthSVA].copy()
    fixedSVADic = collections.Counter(fullSVADF['Fixed'])
    
    notFullLengthSVAs = df.loc[nonFullLengthSVAs].copy()
    reallyFullLengthfromNonFullLengthSVAs = []
    print(speciesName)
    for row in notFullLengthSVAs.index:
        simpleRepeatFlag=0
        aluFlag=0
        svaFlag=0
        myTEList = ast.literal_eval(str(notFullLengthSVAs.at[row,'TE_Hits']))
        svaOrientation='NONE'
        aluOrientation='NONE'
        
        for hit in myTEList:
            goodHit = hit.split()
            if 'Simple_repeat' in str(goodHit[10]):
                simpleRepeatFlag=1
            elif 'Alu' in str(goodHit[10]):
                aluFlag=1
                aluOrientation=str(goodHit[8])
            elif 'SVA' in str(goodHit):
                svaFlag=1
                svaOrientation=str(goodHit[8])

            else:
                continue

        if simpleRepeatFlag ==1 and aluFlag==1 and svaFlag==1:
            if aluOrientation != svaOrientation:
                reallyFullLengthfromNonFullLengthSVAs.append(row)
                #print(myTEList)
                #print('\n')
            else:
                continue
        else:
            continue
    
    print(len(reallyFullLengthfromNonFullLengthSVAs)/len(notFullLengthSVAs), len(reallyFullLengthfromNonFullLengthSVAs))
    #if speciesName == 'PanPan':
    #    break
    
    for element in set(df['TE_Designation']):
        if 'SINE' in element:
            FullLengthList.append([speciesName, fixedAluDic['Yes'], len(df[(df['TE_Designation']==element) & (df['Fixed']=='Yes')]),len(fullLengthAlus), len(df[df['TE_Designation']==element]), 'Cactus_Alu'])
            
        elif 'LINE' in element:
            FullLengthList.append([speciesName, fixedLINEDic['Yes'], len(df[(df['TE_Designation']==element) & (df['Fixed']=='Yes')]),len(fullLengthLines), len(df[df['TE_Designation']==element]), 'Cactus_L1'])
            
        elif 'SVA' in element:
            FullLengthList.append([speciesName, fixedSVADic['Yes'], len(df[(df['TE_Designation']==element) & (df['Fixed']=='Yes')]),len(fullLengthSVA), len(df[df['TE_Designation']==element]), 'Cactus_SVA'])
        else:
            continue
        
    
    #with open('/home/mark/Desktop/TE_Leaves/ls_primates/FullLengthLines_Parithi/Cactus/'+str(speciesName)+'_LS_FL_LINE_INSERTIONS_Cactus_07-08-2024.fasta', 'a+') as outfile:
    #    for row in fullLineDF.index:
    #        if fullLineDF.at[row,'Orientation'] == '+':
    #            outfile.write(">"+str(row)+'\n')
    #            outfile.write(str(fullLineDF.at[row,'Sequence'])+'\n')
    #        else:
    #            sequence = Seq(str(fullLineDF.at[row,'Sequence']))
    #            outfile.write(">"+str(row)+'\n')
    #            outfile.write(str(sequence.reverse_complement())+'\n')
    #outfile.close()

PanTro
0.00980392156862745 1
PanPan
0.04878048780487805 6
PonAbe
0.05172413793103448 3
GorGor
0.013157894736842105 2
PonPyg
0.06329113924050633 5
hs1
0.006493506493506494 1


In [8]:
fullLengthDF = pd.DataFrame(data=FullLengthList, columns=['Species','Fixed_FL_Elements', 'Total_Fixed_Elements','FL_Elements','Total_Elements','DataSet'])

In [10]:
#fullLengthDF.to_csv('/home/mark/Desktop/TE_Leaves/FullLengthElement_Count_WSexChromosomes_07-08-2024.csv')